# Assignment 3 

#### Setup

In [1]:
import pandas as pd
import numpy as np
import random
import time

# Set parameters 
n_inputs = 784
n_outputs = 10
n_hiddens = 16
epochs = 75
learning_rate = 0.005
training_sample_size = 5000

# Import data
training_set = pd.read_csv("Assets/training60000.csv")
training_labels = pd.read_csv("Assets/training60000_labels.csv")
testing_set = pd.read_csv("Assets/testing10000.csv")
testing_labels = pd.read_csv("Assets/testing10000_labels.csv")

# Drop the index columns of the dataframes and convert them to arrays
training_set = training_set.iloc[:,1:785].values
training_labels = training_labels.iloc[:,1].values
testing_set = testing_set.iloc[:,1:785].values
testing_labels = testing_labels.iloc[:,1].values

# Choose random entries from the training set and their corresponding labels
training_set_sample = []#training_set[:1000]
training_labels_sample = []#training_labels[:1000]
for i in range(training_sample_size):
    r = random.randint(0,len(training_set))
    training_set_sample.append(training_set[r])
    np.delete(training_set, r)
    training_labels_sample.append(training_labels[r])
    np.delete(training_labels, r)

#### Initialization 

In [2]:
# Initialize a layer with random weights
def initialize_layer(n_upstream, n_downstream):
    layer = []
    for j in range(n_downstream):
        weights = []
        for i in range(n_upstream + 1): # add 1 to account for the bias
            weights.append(random.uniform(-.5,.5))
        layer.append(weights)
    return layer

# Returns a nested array. Outter array represents the network,
# and each inner array represents a layer in the network.
# The weights are stored within the inner arrays.
def initialize_network(n_inputs, n_hiddens, n_outputs):
    print("----------------------------------------------------------- Initializing network... ")
    hidden_layer = initialize_layer(n_inputs, n_hiddens)
    output_layer = initialize_layer(n_hiddens, n_outputs)
    network = [hidden_layer, output_layer]
    print("----------------------------------------------------------- Initializing complete ")
    return network

#### Training

In [3]:
def sigmoid(x): 
    return (1.0 / (1.0 + np.exp(-x)))
      
def sigmoid_derivative(x): 
    return x * (1 - x)

# Store the outputs of each node. Data is stored in a nested array.
# Parent array has length 2. Index 0 stores the hidden layer nodes,
# index 1 stores the output layer nodes.
# Ex: if we want the output of hidden node 2 -> ouputs[0][1]
def forward_propagate(network, input_vector):
    outputs = []
    # Parse layers in network
    for i in range(len(network)):
        layer = network[i]
        layer_outputs = []
        # Parse nodes in layer, where node[n] is weight n
        for node in layer:
            # Sum of (weights * inputs) coming into node.
            # node[0] is the bias.
            net_input = np.dot(node[1:], input_vector) + node[0]
            node_output = sigmoid(net_input)
            layer_outputs.append(node_output)
        input_vector = layer_outputs
        outputs.append(layer_outputs)
    return outputs

# Return an array containing the errors of the output layer nodes
def oLayer_errors(layer_outputs, predicted):
    errors = []
    # Parse the nodes
    for i in range(len(layer_outputs)):
        output = layer_outputs[i]
        target = predicted[i]
        error = output * (1 - output) * (target - output)
        errors.append(error)
    return errors

# Return an array containing the errors of the hidden layer nodes
def hLayer_errors(network, layer_outputs, output_errors):
    errors = []
    weights = network[0]
    # Parse the nodes
    for i in range(len(layer_outputs)):
        output = layer_outputs[i]
        # handles sigma(W_kh * error_k)
        summation = 0
        for j in range(len(output_errors)):
            summation += (weights[i][j] * output_errors[j])
        # Full error term equation for hidden nodes
        error = output * (1 - output) * summation
        errors.append(error)
    return errors

# Returns a nested array "error_terms".
# error_terms[0] = errors of nodes in hidden layer,
# error_terms[1] = errors of nodes in output layer.
def backward_propagate(network, all_node_outputs, predicted):
    error_terms = []
    output_layer_o = all_node_outputs[1]
    output_errors = oLayer_errors(output_layer_o, predicted)
    hidden_layer_o = all_node_outputs[0]
    hidden_layer_w = network[1] # We want the weights coming from hidden layer into output layer
    hidden_errors = hLayer_errors(network, hidden_layer_o, output_errors)

    error_terms.append(hidden_errors)
    error_terms.append(output_errors)
    return error_terms
    
def update_layer(network, layer_n, lr, errors, inputs):
    new_layer = []
    layer = network[layer_n]
    errors = errors[layer_n]
    for n in range(len(layer)):
        node = layer[n]
        new_node = []
        for w in range(len(node)):
            weight = node[w]
            delta = lr * inputs[node.index(weight) - 1] * errors[layer.index(node)] 
            new_weight = weight + delta
            new_node.append(new_weight)
        new_layer.append(new_node)
    return new_layer

def update_weights(network, l_rate, node_outputs, errors, input_vector):
    new_hidden_weights = update_layer(network, 0, l_rate, errors, input_vector)
    new_output_weights = update_layer(network, 1, l_rate, errors, node_outputs[0])
    new_network = []
    new_network.append(new_hidden_weights)
    new_network.append(new_output_weights)
    return new_network
    
# The main function for the training algorithm.
def train_network(network, training_examples, training_labels, epochs, learning_rate):
    print("----------------------------------------------------------- Training network.... ")
    n_outputs = len(network[1])
    for epoch in range(epochs):
        t0 = time.time()
        for i in range(len(training_examples)):
            if (i+1) % 1250 == 0:
                print("\tModel trained on",i+1,"of",len(training_examples),"instances")
            target_value = training_labels[i]
            input_vector = training_examples[i]
            all_node_outputs = forward_propagate(network, input_vector)
            output_vector = all_node_outputs[1]
            predicted_vector = []
            for n in range(n_outputs):
                predicted_vector.append(0.01)
            predicted_vector[target_value] = 0.99
            errors = backward_propagate(network, all_node_outputs, predicted_vector)
            network = update_weights(network, learning_rate, all_node_outputs, errors, input_vector)
        print("\tEpoch",epoch+1,"complete.")
    print("----------------------------------------------------------- Training complete ")
    return network


#### Evalutation

In [4]:
def evaluate_network(network, test_examples, test_labels):
    print("----------------------------------------------------------- Evaluating network... ")
    n_correct = 0
    n_incorrect = 0
    for i in range(len(test_examples)):
        input_vector = test_examples[i]
        output_vector = forward_propagate(network, input_vector)[1]
        prediction = np.argmax(output_vector)
        if prediction == test_labels[i]:
            n_correct += 1
        if prediction != test_labels[i]:
            n_incorrect += 1
    print("----------------------------------------------------------- Evaluation complete ")
    print("Results:")
    print("\tCorrect classifications:\t",n_correct)
    print("\tIncorrect classifications:\t",n_incorrect)
    print("\tAccuracy:\t\t\t","{:.2%}".format(n_correct / len(test_examples)))

#### Visualizion

In [5]:
def print_layer(network, layer):
    layer_array = []
    if layer == "hidden":
        layer_array = network[0]
    if layer == "output":
        layer_array = network[1]
    #else:
    #    print("Error. Invalid value for 'layer' parameter in function print_layer(network,layer)")
    for n in range(len(layer_array)):
        node = layer_array[n]
        print()
        print("weights going into",layer,"node",n)
        for w in range(len(node)):
            weight = node[w]
            print("weight",w,":",weight)

def print_weights(network):
    print_layer(network,"hidden")
    #print("# of weights going into output layer:", len(network[0]))
    print_layer(network,"output")
    
def print_params():
    print("Chosen parameters:")
    print("\tInput nodes:\t\t",n_inputs)
    print("\tHidden nodes:\t\t",n_hiddens)
    print("\tOutput nodes:\t\t",n_outputs)
    print("\tEpochs:\t\t\t",epochs)
    print("\tLearning Rate:\t\t",learning_rate)
    print("\tTraining samples:\t",training_sample_size)
    print()

#### Run the program

In [6]:
print_params()
network = initialize_network(n_inputs, n_hiddens, n_outputs)
start = time.time()
trained = train_network(network, training_set_sample, training_labels_sample, epochs, learning_rate)
stop = time.time()
total_time = stop-start
print("\tTraining time:", total_time)
evaluate_network(trained, testing_set, testing_labels)

Chosen parameters:
	Input nodes:		 784
	Hidden nodes:		 16
	Output nodes:		 10
	Epochs:			 75
	Learning Rate:		 0.005
	Training samples:	 5000

----------------------------------------------------------- Initializing network... 
----------------------------------------------------------- Initializing complete 
----------------------------------------------------------- Training network.... 
	Model trained on 1250 of 5000 instances
	Model trained on 2500 of 5000 instances
	Model trained on 3750 of 5000 instances
	Model trained on 5000 of 5000 instances
	Epoch 1 complete.
	Model trained on 1250 of 5000 instances
	Model trained on 2500 of 5000 instances
	Model trained on 3750 of 5000 instances
	Model trained on 5000 of 5000 instances
	Epoch 2 complete.
	Model trained on 1250 of 5000 instances
	Model trained on 2500 of 5000 instances
	Model trained on 3750 of 5000 instances
	Model trained on 5000 of 5000 instances
	Epoch 3 complete.
	Model trained on 1250 of 5000 instances
	Model trained o

	Model trained on 3750 of 5000 instances
	Model trained on 5000 of 5000 instances
	Epoch 43 complete.
	Model trained on 1250 of 5000 instances
	Model trained on 2500 of 5000 instances
	Model trained on 3750 of 5000 instances
	Model trained on 5000 of 5000 instances
	Epoch 44 complete.
	Model trained on 1250 of 5000 instances
	Model trained on 2500 of 5000 instances
	Model trained on 3750 of 5000 instances
	Model trained on 5000 of 5000 instances
	Epoch 45 complete.
	Model trained on 1250 of 5000 instances
	Model trained on 2500 of 5000 instances
	Model trained on 3750 of 5000 instances
	Model trained on 5000 of 5000 instances
	Epoch 46 complete.
	Model trained on 1250 of 5000 instances
	Model trained on 2500 of 5000 instances
	Model trained on 3750 of 5000 instances
	Model trained on 5000 of 5000 instances
	Epoch 47 complete.
	Model trained on 1250 of 5000 instances
	Model trained on 2500 of 5000 instances
	Model trained on 3750 of 5000 instances
	Model trained on 5000 of 5000 instance